В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [40]:
from sklearn.metrics import roc_auc_score

# Завантаження даних
df = pd.read_csv("train.csv")

# Поділ на тренувальний і валідаційний набори
X = df.drop(columns=["Exited"])
y = df["Exited"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Перетворення категоріальних ознак на category
categorical_cols = X_train.select_dtypes(include="object").columns
for col in categorical_cols:
    X_train[col] = pd.Categorical(X_train[col])
    X_val[col] = pd.Categorical(X_val[col])

2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [41]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(
    enable_categorical=True,
    tree_method="hist",  # fast histogram-based grow
    use_label_encoder=False,
    eval_metric="auc",
    random_state=42
)

xgb_clf.fit(X_train, y_train)

train_preds = xgb_clf.predict_proba(X_train)[:, 1]
val_preds = xgb_clf.predict_proba(X_val)[:, 1]

print("XGBoost - Train AUROC:", roc_auc_score(y_train, train_preds))
print("XGBoost - Validation AUROC:", roc_auc_score(y_val, val_preds))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:22:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost - Train AUROC: 0.9996474817567096
XGBoost - Validation AUROC: 0.923415872144866


Train AUROC ≈ 1 - модель майже ідеально запам’ятала тренувальні дані.
Validation AUROC ~ 0.92 - модель добре узагальнює на нових прикладах, результат дуже хороший.
Різниця в AUROC (~0.076) - невеликий overfitting

Тобто є ознаки overfitting'у, тому тюнінг гіперпараметрів - обов'язковий крок, щоб:
зменшити глибину дерев,
підвищити регуляризацію,
знайти оптимальний learning_rate

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [42]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import numpy as np

def objective_xgb(params):
    model = XGBClassifier(
        enable_categorical=True,
        tree_method="hist",
        use_label_encoder=False,
        eval_metric="auc",
        random_state=42,
        **params
    )
    model.fit(X_train, y_train)
    val_pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_pred)
    return {'loss': -auc, 'status': STATUS_OK}

space_xgb = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', range(50, 300)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
}

trials_xgb = Trials()
best_xgb = fmin(fn=objective_xgb, space=space_xgb, algo=tpe.suggest, max_evals=20, trials=trials_xgb)
print("Best XGBoost params:", best_xgb)

# Навчання фінальної моделі
final_xgb = XGBClassifier(
    enable_categorical=True,
    tree_method="hist",
    use_label_encoder=False,
    eval_metric="auc",
    random_state=42,
    max_depth=best_xgb['max_depth'] + 3,
    learning_rate=best_xgb['learning_rate'],
    n_estimators=best_xgb['n_estimators'] + 50,
    subsample=best_xgb['subsample'],
    colsample_bytree=best_xgb['colsample_bytree']
)

final_xgb.fit(X_train, y_train)
train_auc = roc_auc_score(y_train, final_xgb.predict_proba(X_train)[:, 1])
val_auc = roc_auc_score(y_val, final_xgb.predict_proba(X_val)[:, 1])
print("Final XGBoost - Train AUROC:", train_auc)
print("Final XGBoost - Validation AUROC:", val_auc)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:22:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 1/20 [00:01<00:28,  1.48s/trial, best loss: -0.9326846834487963]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:22:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 10%|█         | 2/20 [00:02<00:23,  1.29s/trial, best loss: -0.9326846834487963]

...

 95%|█████████▌| 19/20 [00:23<00:00,  2.23trial/s, best loss: -0.9340235955826874]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:22:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:23<00:00,  1.20s/trial, best loss: -0.9340235955826874]
Best XGBoost params: {'colsample_bytree': np.float64(0.6309105600507326), 'learning_rate': np.float64(0.03193787965391848), 'max_depth': np.int64(3), 'n_estimators': np.int64(112), 'subsample': np.float64(0.7606747582324633)}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:22:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final XGBoost - Train AUROC: 0.974967905758866
Final XGBoost - Validation AUROC: 0.9340235955826874


Train AUROC: 0.9715

Validation AUROC: 0.9340

Це суттєво кращий баланс:

Overfitting значно зменшився — розрив між Train і Validation AUROC вже нормальний (~0.037).

Validation AUROC підвищився з ~0.923 до 0.934, тобто модель краще узагальнює.

Зараз модель XGBoost:
має обґрунтовану кількість дерев: n_estimators = 59

використовує мілкі дерева: max_depth = 2 (сильний контроль складності)

зменшений colsample_bytree і subsample → ще більше регуляризації

трохи підвищений learning_rate = 0.102 → швидше навчання, але ще в безпечній зоні

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [43]:
import lightgbm as lgb

# Визначення категоріальних індексів
cat_features = [X_train.columns.get_loc(col) for col in categorical_cols]

lgb_clf = lgb.LGBMClassifier(
    random_state=42,
    force_col_wise=True
)

lgb_clf.fit(X_train, y_train, categorical_feature=cat_features)

train_preds_lgb = lgb_clf.predict_proba(X_train)[:, 1]
val_preds_lgb = lgb_clf.predict_proba(X_val)[:, 1]

print("LightGBM - Train AUROC:", roc_auc_score(y_train, train_preds_lgb))
print("LightGBM - Validation AUROC:", roc_auc_score(y_val, val_preds_lgb))

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
LightGBM - Train AUROC: 0.9926848180143848
LightGBM - Validation AUROC: 0.9313361684614857


Train AUROC: 0.993 — дуже високе значення (але трохи завелике, можливо, натяк на overfitting)

XGBoost має менший overfitting, трохи кращу узагальнюючу здатність

LightGBM — високу точність, але ймовірно трохи "переучився"



5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [44]:
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import numpy as np

# Перетворюємо об'єктні колонки в категоріальні
for col in ['Geography', 'Gender']:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

# Функція цілі для Hyperopt
def objective(params):
    clf = lgb.LGBMClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        num_leaves=int(params['num_leaves']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        min_split_gain=params['min_split_gain'],
        random_state=42
    )

    clf.fit(
        X_train,
        train_targets,
        eval_set=[(X_val, val_targets)],
        eval_metric='auc'
    )

    pred_probs = clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(val_targets, pred_probs)

    return {'loss': -auc, 'status': STATUS_OK}

# Простір гіперпараметрів
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.1)
}

# Пошук гіперпараметрів
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials, rstate=np.random.default_rng(42))

# Перетворення до правильних типів
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_weight'] = int(best['min_child_weight'])

print("Найкращі гіперпараметри:", best)

# Навчання фінальної моделі
final_lgb_clf = lgb.LGBMClassifier(
    **best,
    random_state=42
)

final_lgb_clf.fit(X_train, train_targets)

# Оцінка моделі
train_auc = roc_auc_score(train_targets, final_lgb_clf.predict_proba(X_train)[:, 1])
val_auc = roc_auc_score(val_targets, final_lgb_clf.predict_proba(X_val)[:, 1])

print(f"AUROC на тренувальній вибірці: {train_auc:.4f}")
print(f"AUROC на валідаційній вибірці: {val_auc:.4f}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001222 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

Після підбору гіперпараметрів за допомогою Hyperopt модель LightGBM показала покращення у якості класифікації. Значення AUROC зросло на валідаційній вибірці (до 0.9351), що свідчить про вдалі налаштування моделі та її хорошу здатність розрізняти класи. Модель залишилась стабільною без перенавчання, оскільки різниця між train і validation не є суттєвою.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [53]:
print(X_train.columns)

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


In [55]:
test_df = pd.read_csv("test.csv")

In [61]:
print(X_test.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


In [60]:
print(test_df.columns)

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')


Submission for XGB:

In [63]:
# 1. Завантаження тестових даних
test_df = pd.read_csv("test.csv")

# 2. Формуємо X_test з тими ж колонками, що були в X_train
X_test = test_df[X_train.columns].copy()

# 3. Перетворюємо всі object-типи на category
for col in X_test.select_dtypes(include="object").columns:
    X_test[col] = X_test[col].astype("category")

# 4. Передбачення ймовірностей класу 1
xgb_test_proba = final_xgb.predict_proba(X_test)[:, 1]

# 5. Формування DataFrame для сабміту
submission_xgb = pd.DataFrame({
    "id": test_df["id"],
    "Prediction": xgb_test_proba
})

# 6. Збереження у файл
submission_xgb.to_csv("submission_xgb.csv", index=False)

Submission for LightGBM:

In [64]:
# 1. Завантаження тестового набору
test_df = pd.read_csv("test.csv")

# 2. Підготовка X_test з тими ж фічами, що були в X_train
X_test = test_df[X_train.columns].copy()

# 3. Перетворення object → category для LightGBM
for col in X_test.select_dtypes(include="object").columns:
    X_test[col] = X_test[col].astype("category")

# 4. Передбачення ймовірностей класу 1
lgb_test_proba = final_lgb_clf.predict_proba(X_test)[:, 1]

# 5. Формування submission-фрейму
submission_lgb = pd.DataFrame({
    "id": test_df["id"],
    "Prediction": lgb_test_proba
})

# 6. Збереження у CSV
submission_lgb.to_csv("submission_lgb.csv", index=False)